In [12]:
import cv2
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input as resnet_preprocess_input
from keras.applications.vgg19 import preprocess_input as vgg19_preprocess_input
from keras.layers import AveragePooling2D, Dense, Flatten, Dropout
from keras.models import Model, load_model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import keras
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import argparse
import os

In [13]:
DIR_TEST = "dataset-cropped/Test/"

In [29]:
images = []
labels = []

df = pd.read_csv("dataset-cropped/manual_ans.csv")

for tps in os.listdir(DIR_TEST):
    for paslon in os.listdir(os.path.join(DIR_TEST, tps)):
        for section in os.listdir(os.path.join(DIR_TEST, tps, paslon)):
            path = os.path.join(DIR_TEST, tps, paslon, section)

            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (40, 240))
            img = cv2.GaussianBlur(img, (5, 5), 0)
            # imt = cv2.medianBlur(img, 5)

            th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 19, 2)
            # cv2.floodFill(th, None, (0, 0), 123, )
            
            th = th / 255.0
            images.append(th)
    
    curr_tps = df[df['TPS'] == tps]
    for col in (['Paslon_1', 'Paslon_2', 'Paslon_3']):
        label_str = '{:03d}'.format(curr_tps[col].values[0])
        labels.extend([int(n) for n in label_str])

In [32]:
def evaluation(model_name, images, labels):
    model = load_model(f'models/{model_name}')
    tmp_images = np.array(images)
    tmp_labels = np.array(labels)

    tmp_images = np.repeat(tmp_images[..., np.newaxis], 3, -1)
    if model_name == 'ResNet50PreprocessInput' :
        tmp_images = resnet_preprocess_input(tmp_images)
    elif model_name == 'VGG19' :
        tmp_images = vgg19_preprocess_input(tmp_images)

    tmp_labels = keras.utils.to_categorical(tmp_labels, num_classes=10)

    loss, acc = model.evaluate(tmp_images, tmp_labels)
    print(f'Loss: {loss}')
    print(f'Accuracy: {acc}')


RESNET50

In [35]:
evaluation('ResNet50PreprocessInput', images, labels)

51/51 [==============================] - 3s 28ms/step - loss: 0.0643 - accuracy: 0.9858
Loss: 0.06432805210351944
Accuracy: 0.9858024716377258


In [36]:
evaluation('ResNet50', images, labels)

51/51 [==============================] - 2s 27ms/step - loss: 0.0717 - accuracy: 0.9858
Loss: 0.07172521203756332
Accuracy: 0.9858024716377258


VGG19

In [24]:
evaluation('VGG19', images, labels)

51/51 [==============================] - 2s 33ms/step - loss: 0.1769 - accuracy: 0.9710
Loss: 0.17688040435314178
Accuracy: 0.9709876775741577
